In [15]:
#####Gaussian Process Spatio Temporal point model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.matlib
import scipy.linalg
import itertools
import calendar
from datetime import datetime
from datetime import timedelta
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [16]:
##乱数を生成する関数を設定
#多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

#多変量正規分布の乱数を生成する関数
def rmvnorm(mu, Cov, hh, k):
    s = mu + np.random.normal(0, 1, hh*k).reshape(hh, k)
    P = np.linalg.cholesky(Cov)
    y = np.dot(P, s.T).T
    return y

#任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper, pattern):
    #相関行列の初期値を定義する
    if pattern==1:
        Prob = np.abs(lower) / (np.abs(lower) + np.abs(upper))
        z = np.random.binomial(1, Prob, col*col); z[z==0] = -1
        cov_vec = np.random.beta(1.0, 3.0, col*col) * z   #相関係数の乱数ベクトルを作成
    else:
        cov_vec = np.random.uniform(lower, upper, col*col)   #相関係数の乱数ベクトルを作成
    rho = np.tril(cov_vec.reshape(col, col), k=-1)   #乱数ベクトルを下三角行列化
    Sigma = (rho + rho.T) + np.diag(np.repeat(1, col))   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = np.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(col):
        if eigen_val[i] < 0:
            eigen_val[i] = np.random.uniform(eigen_lower, eigen_upper, 1)
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(np.power(np.diag(Sigma), 0.5)[:, np.newaxis], np.power(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

#相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

#分散共分散行列から相関行列に変換する関数
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [46]:
####データの生成####
##データの設定
weeks = 7
k = 10
place = 3000
category = 150
max_m = 5
m = np.random.poisson(1.5, place)
m[m==0] = 1; m[m > max_m] = max_m 
Lambda = np.random.gamma(75, 1/0.5, place)
pt = np.random.poisson(Lambda, place)
N = np.sum(pt)

In [47]:
##idとインデックスを作成
#idの作成
d_id = np.repeat(np.arange(place), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(place)])))

#インデックスの設定
d_list = [i for i in range(place)]
for i in range(place):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")

In [48]:
##カテゴリー割当を定義
#トピックを生成
s = 30
prob = np.random.dirichlet(np.repeat(0.75, s), 1).reshape(-1)
phi = np.random.dirichlet(np.repeat(0.5, category), s)
z = np.dot(np.random.multinomial(1, prob, N), np.arange(s))

#多項分布からカテゴリーを生成
category_id = np.full((place, max_m), category)
for i in range(place):
    while True:
        generate = np.random.multinomial(m[i], phi[z[i], ], 1)
        if np.max(generate)==1:
            new_generate = generate * np.arange(category)
            category_id[i, np.arange(m[i])] = new_generate[generate > 0]
            break

In [54]:
category_id

array([[ 97, 140, 150, 150, 150],
       [111, 150, 150, 150, 150],
       [108, 110, 135, 150, 150],
       ...,
       [ 69, 150, 150, 150, 150],
       [ 13, 150, 150, 150, 150],
       [ 30, 123, 150, 150, 150]])

In [41]:
category_id

array([[150, 150, 150, 150, 150],
       [150, 150, 150, 150, 150],
       [150, 150, 150, 150, 150],
       ...,
       [150, 150, 150, 150, 150],
       [150, 150, 150, 150, 150],
       [150, 150, 150, 150, 150]])